In [ ]:
import tkinter as tk
from tkinter import messagebox, ttk
import sqlite3
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError

# Email and Slack Configuration
EMAIL_ADDRESS = "avishalbharadwaj.com"
EMAIL_PASSWORD = "password"  
SLACK_TOKEN = "your_slack_bot_token"
DEFAULT_SLACK_CHANNEL = "Group_2_538"     

# Send email notifications
def send_email(subject, body):
    try:
        msg = MIMEMultipart()
        msg['From'] = EMAIL_ADDRESS
        msg['To'] = EMAIL_ADDRESS  # Send to yourself or others
        msg['Subject'] = subject
        msg.attach(MIMEText(body, 'plain'))

        with smtplib.SMTP('smtp.gmail.com', 587) as server:
            server.starttls()
            server.login(EMAIL_ADDRESS, EMAIL_PASSWORD)
            server.send_message(msg)
        print("Email sent successfully!")
    except Exception as e:
        print(f"Failed to send email: {e}")

# Send Slack notifications
def send_slack_message(message, channel=DEFAULT_SLACK_CHANNEL):
    try:
        client = WebClient(token=SLACK_TOKEN)
        response = client.chat_postMessage(channel=channel, text=message)
        print("Slack notification sent successfully!")
    except SlackApiError as e:
        print(f"Failed to send Slack message: {e.response['error']}")

# Database setup
def setup_database():
    conn = sqlite3.connect('inventory.db')
    cursor = conn.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS inventory (
                        id INTEGER PRIMARY KEY,
                        product_name TEXT NOT NULL,
                        quantity INTEGER NOT NULL,
                        price_per_unit REAL NOT NULL)''')
    conn.commit()
    conn.close()

# Add product to the inventory
def add_product():
    product_name = entry_product_name.get()
    quantity = entry_quantity.get()
    price = entry_price.get()
    slack_channel = entry_slack_channel.get() or DEFAULT_SLACK_CHANNEL

    if not product_name or not quantity.isdigit() or not price:
        messagebox.showwarning("Input Error", "Please enter valid product details!")
        return

    conn = sqlite3.connect('inventory.db')
    cursor = conn.cursor()
    cursor.execute("INSERT INTO inventory (product_name, quantity, price_per_unit) VALUES (?, ?, ?)",
                   (product_name, int(quantity), float(price)))
    conn.commit()
    conn.close()
    messagebox.showinfo("Success", f"Added {product_name} to inventory.")

    # Send notification
    send_email("New Product Added", f"Product: {product_name}, Quantity: {quantity}, Price: {price}")
    send_slack_message(f"New Product Added: {product_name}, Quantity: {quantity}, Price: {price}", channel=slack_channel)

    entry_product_name.delete(0, tk.END)
    entry_quantity.delete(0, tk.END)
    entry_price.delete(0, tk.END)
    entry_slack_channel.delete(0, tk.END)
    view_inventory()

# View inventory items
def view_inventory():
    tree.delete(*tree.get_children())
    conn = sqlite3.connect('inventory.db')
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM inventory")
    rows = cursor.fetchall()
    conn.close()
    for row in rows:
        tree.insert('', tk.END, values=row)

# Update product quantity
def update_quantity():
    selected_item = tree.focus()
    if not selected_item:
        messagebox.showwarning("Selection Error", "Please select an item to update!")
        return

    new_quantity = entry_quantity.get()
    slack_channel = entry_slack_channel.get() or DEFAULT_SLACK_CHANNEL

    if not new_quantity.isdigit():
        messagebox.showwarning("Input Error", "Please enter a valid quantity!")
        return

    item_id = tree.item(selected_item)['values'][0]
    product_name = tree.item(selected_item)['values'][1]
    conn = sqlite3.connect('inventory.db')
    cursor = conn.cursor()
    cursor.execute("UPDATE inventory SET quantity = ? WHERE id = ?", (int(new_quantity), item_id))
    conn.commit()
    conn.close()
    messagebox.showinfo("Success", "Quantity updated.")

    # Send notification if quantity is low
    if int(new_quantity) < 10:
        send_email("Low Stock Alert", f"Product '{product_name}' is running low with only {new_quantity} units left.")
        send_slack_message(f"Low Stock Alert: Product '{product_name}' has only {new_quantity} units remaining.", channel=slack_channel)

    view_inventory()

# Delete a product
def delete_product():
    selected_item = tree.focus()
    if not selected_item:
        messagebox.showwarning("Selection Error", "Please select an item to delete!")
        return

    item_id = tree.item(selected_item)['values'][0]
    product_name = tree.item(selected_item)['values'][1]
    slack_channel = entry_slack_channel.get() or DEFAULT_SLACK_CHANNEL

    conn = sqlite3.connect('inventory.db')
    cursor = conn.cursor()
    cursor.execute("DELETE FROM inventory WHERE id = ?", (item_id,))
    conn.commit()
    conn.close()
    messagebox.showinfo("Success", "Product deleted.")

    # Send notification
    send_email("Product Deleted", f"Product '{product_name}' has been removed from the inventory.")
    send_slack_message(f"Product Deleted: '{product_name}' has been removed from the inventory.", channel=slack_channel)

    view_inventory()

# Main GUI
root = tk.Tk()
root.title("Inventory Management System")
root.geometry("600x600")

setup_database()

# Input fields
frame_input = tk.Frame(root)
frame_input.pack(pady=10)

tk.Label(frame_input, text="Product Name").grid(row=0, column=0, padx=5)
entry_product_name = tk.Entry(frame_input)
entry_product_name.grid(row=0, column=1)

tk.Label(frame_input, text="Quantity").grid(row=1, column=0, padx=5)
entry_quantity = tk.Entry(frame_input)
entry_quantity.grid(row=1, column=1)

tk.Label(frame_input, text="Price per Unit").grid(row=2, column=0, padx=5)
entry_price = tk.Entry(frame_input)
entry_price.grid(row=2, column=1)

tk.Label(frame_input, text="Slack Channel (optional)").grid(row=3, column=0, padx=5)
entry_slack_channel = tk.Entry(frame_input)
entry_slack_channel.grid(row=3, column=1)

# Buttons
frame_buttons = tk.Frame(root)
frame_buttons.pack(pady=10)

tk.Button(frame_buttons, text="Add Product", command=add_product).grid(row=0, column=0, padx=5)
tk.Button(frame_buttons, text="Update Quantity", command=update_quantity).grid(row=0, column=1, padx=5)
tk.Button(frame_buttons, text="Delete Product", command=delete_product).grid(row=0, column=2, padx=5)

# Inventory table
tree = ttk.Treeview(root, columns=('ID', 'Product Name', 'Quantity', 'Price per Unit'), show='headings')
tree.heading('ID', text='ID')
tree.heading('Product Name', text='Product Name')
tree.heading('Quantity', text='Quantity')
tree.heading('Price per Unit', text='Price per Unit')
tree.pack(fill=tk.BOTH, expand=True)

view_inventory()

root.mainloop()